In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer


## Quadratic Layer

In [2]:
class SimpleQuadratic(Layer):
    def __init__(self,units=32,activation=None): 
        super(SimpleQuadratic,self).__init__()
        self.activation=tf.keras.activations.get(activation)
        self.units=units
    def build(self,input_shape):
        a_int=tf.random_normal_initializer()
        b_int=tf.random_normal_initializer()
        c_int=tf.zeros_initializer()
        self.a=tf.Variable(name='a',
                          initial_value=a_int(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
        self.b=tf.Variable(name='b',
                          initial_value=b_int(shape=(input_shape[-1],self.units),dtype='float32')
                          ,trainable=True)
        self.c=tf.Variable(name='c',
                          initial_value=c_int(shape=(self.units,),dtype='float32')
                          ,trainable=True)
        
    def call(self,inputs):
        return self.activation(tf.matmul(tf.math.square(inputs),self.a)+tf.matmul(inputs,self.b)+self.c) 

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
   tf.keras.layers.Flatten(input_shape=(28, 28)),
   SimpleQuadratic(128, activation='relu'),
   tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(10, activation='softmax')
 ])

model.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 2s 864us/step - loss: 0.2651 - accuracy: 0.9219
Epoch 2/5
1875/1875 [==============================] - 2s 825us/step - loss: 0.1310 - accuracy: 0.9609
Epoch 3/5
1875/1875 [==============================] - 2s 819us/step - loss: 0.0998 - accuracy: 0.9696
Epoch 4/5
1875/1875 [==============================] - 2s 816us/step - loss: 0.0805 - accuracy: 0.9748
Epoch 5/5
313/313 [==============================] - 0s 622us/step - loss: 0.0726 - accuracy: 0.9779


[0.07256614416837692, 0.9779000282287598]

In [9]:
# inherit from the Model base class
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        '''initializes the instance attributes'''
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = Dense(units, activation=activation)
        self.main_output = Dense(1)
        self.aux_output = Dense(1)

    def call(self, inputs):
        '''defines the network architecture'''
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        
        return main_output, aux_output

## Resnet_Model
#### Implement Model Subclass

In [10]:
class IdentityBlock(tf.keras.Model):
    def __init__(self,filters,kernel_size):
        super().__init__(name='')
        self.conv1=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
        self.conv2=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
        self.bn1=tf.keras.layers.BatchNormalization()
        self.act=tf.keras.layers.Activation('relu')
        self.add=tf.keras.layers.Add()
    def call(self,input_tensor):
        x=self.conv1(input_tensor)
        x=self.bn1(x)
        x=self.act(x)
        x=self.conv2(x)
        x=self.bn1(x)
        x=self.add([x,input_tensor])
        x=self.act(x)
        return x

In [11]:
class Resnet(tf.keras.Model):
    def __init__(self,num_classes):
        super().__init__()
        self.con=tf.keras.layers.Conv2D(64,7,padding='same')
        self.bn=tf.keras.layers.BatchNormalization()
        self.act=tf.keras.layers.Activation('relu')
        self.max_pol=tf.keras.layers.MaxPool2D((3,3))
        self.idbl1= IdentityBlock(64,3)
        self.idblb= IdentityBlock(64,3)
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.quad=SimpleQuadratic(32,activation='relu')
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
    def call(self,inputs):
        x=self.con(inputs)
        x=self.bn(x)
        x=self.act(x)
        x=self.max_pol(x)
        x=self.idbl1(x)
        x=self.idblb(x)
        x=self.global_pool(x)
        x=self.quad(x)
        return self.classifier(x)

In [13]:
# utility function to normalize the images and return (image, label) pairs.
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

# create a ResNet instance with 10 output units for MNIST
resnet = WideAndDeepModel(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# load and preprocess the dataset
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)

# train the model.
resnet.fit(dataset, epochs=1)

NameError: name 'Dense' is not defined

### Callbacks

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training after reaching 60 percent accuracy

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check accuracy
    if(logs.get('loss') < 0.4):
        print("\nLoss is lower than 0.4 so cancelling training!")
        self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

In [ ]:
%%latex